1. Introduction

# Model Development for NetTrafficGuard

In this notebook, we will develop machine learning models to analyze and enhance network traffic security. We'll use three processed datasets for our analysis:
1. **CICIDS2017_processed.csv**
2. **KDDCup1999_processed.csv**
3. **NSL-KDD_processed.csv**

The primary steps include:
- Loading and merging the datasets
- Feature engineering
- Splitting the data into training and test sets
- Building and evaluating various machine learning models


2. Load and Prepare Data


In [11]:
import pandas as pd

# Load datasets
cicids2017_df = pd.read_csv('E:\\Hackatone Project\\NetTrafficGuard\\data\\processed\\processedCICIDS2017_Cleaned.csv')
kddcup1999_df = pd.read_csv('E:\\Hackatone Project\\NetTrafficGuard\\data\\processed\\KDDCup1999_Cleaned.csv')
nsl_kdd_df = pd.read_csv('E:\\Hackatone Project\\NetTrafficGuard\\data\\processed\\NSL-KDD_Cleaned.csv')

# Display basic information
print("CICIDS2017 DataFrame:")
print(cicids2017_df.info())
print(cicids2017_df.head())

print("\nKDDCup1999 DataFrame:")
print(kddcup1999_df.info())
print(kddcup1999_df.head())

print("\nNSL-KDD DataFrame:")
print(nsl_kdd_df.info())
print(nsl_kdd_df.head())


CICIDS2017 DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119612 entries, 0 to 119611
Data columns (total 85 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Flow ID                     119612 non-null  object 
 1   Src IP                      119612 non-null  object 
 2   Src Port                    119612 non-null  float64
 3   Dst IP                      119612 non-null  object 
 4   Dst Port                    119612 non-null  float64
 5   Protocol                    119612 non-null  float64
 6   Timestamp                   119612 non-null  object 
 7   Flow Duration               119612 non-null  float64
 8   Total Fwd Packet            119612 non-null  float64
 9   Total Bwd packets           119612 non-null  float64
 10  Total Length of Fwd Packet  119612 non-null  float64
 11  Total Length of Bwd Packet  119612 non-null  float64
 12  Fwd Packet Length Max       119612 non-null  float